# Project Title : Analyzing CIA Factbook Data Using SQL







## Table of Contents
* Introduction

* Overview of the data

* Exploratory data analysis


## Introduction

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

* population — the global population.

* population_growth — the annual population growth rate, as a percentage.

* area — the total land and water area.

In this project, we'll use SQL in Jupyter Notebook to analyze data from this database. You can download the SQLite factbook.db database [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db)

We'll use the code in the code cell below to connect this Jupyter Notebook to our database file

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

The connection was successful

## Overview of the Data

We start by exploring the data. We shall add  '%%sql' to run SQL queries.

In [3]:
%%sql
SELECT *
    FROM facts
    LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


**Here are the descriptions of some of the columns from the table above.**

* name — the name of the country.

* area — the country's total area (both land and water).

* area_land — the country's land area in square kilometers.

* area_water — the country's water area in square kilometers.

* population — the country's population.

* population_growth — the country's population growth as a percentage.

* birth_rate — the country's birth rate, or the number of births per year per 1,000 people.

* death_rate — the country's death rate, or the number of death per year per 1,000 people.

# Exploratory data analysis

## Summary Statistics

Let us calculate some summary statistics and look for any outlier countries.

In [4]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics:

* There's a country with a population of 0

* There's a country with a population of 7256490011 (or more than 7.2 billion people)

## Exploring Outliers

Let's use subqueries to zoom in on just these countries without using the specific values. 

First, we will write a query that returns the countries with the minimum population.

In [5]:
%%sql
SELECT name, MIN(population)
FROM facts;

Done.


name,MIN(population)
Antarctica,0


It appears that there is a row for Antartica in the table, which explains the 0 population. Antarctica is a unique continent in that it does not have a native human population. 

Let us run a query that returns countries with the maximum population.

In [6]:
%%sql
SELECT name, MAX(population)
FROM facts;

Done.


name,MAX(population)
World,7256490011


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion.

Now that we know this, we should recalculate the summary statistics, excluding the row for the whole world this time around.

## Exploring Average Population and Area

In [7]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
    FROM facts
    WHERE name <> 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


The result shows that the maximum population is about 1.4 billion!

### Let us look into the average population and average area

In [8]:
%%sql
SELECT AVG(population), AVG(area)
    FROM facts
    WHERE name <> 'World';

Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


The table shows that the average population is about 32 million and the average area is 555 thousand square kilometers.

## Finding Densely Populated Countries

Now we'll build on the query we wrote previously to find countries that are densely populated. We'll identify countries that have the following:

* Above-average values for population.

* Below-average values for area.

In [9]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts
                   WHERE name <> 'World')
    AND area < (SELECT AVG(area)
               FROM facts
               WHERE name <> 'World');

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


From the table above, we can see countries that appear to be densely populated. So we have confidence in our results because some of these countries are popularly known to be densely populated. 

## Country that is most populated

In [10]:
%%sql
SELECT name, MAX(population)
FROM facts
WHERE name <> 'World';

Done.


name,MAX(population)
China,1367485388


It appears that the most populated country is china.

## Country with the highest growth rate

In [11]:
%%sql
SELECT name, MAX(population_growth)
FROM facts
WHERE name <> 'World';

Done.


name,MAX(population_growth)
South Sudan,4.02


The country with the highest growth rate is South Sudan

## Countries that have the highest ratios of water to land

In [18]:
%%sql
SELECT name, CAST(area_water AS Float)/area_land AS water_land_ratio
FROM facts
ORDER BY water_land_ratio DESC
LIMIT 10

Done.


name,water_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


The table above shows ten countries with the highest ratio of water to land.

## Countries that have more water than land

In [21]:
%%sql
SELECT name, area_water, area_land
FROM facts
WHERE name <> 'World' AND (SELECT area_water > area_land);

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


British Indian Ocean Territory and Virgin Islands have more water than land.

## Countries that have higher death rate than birth rate

In [23]:
%%sql
SELECT name, birth_rate, death_rate
FROM facts
WHERE death_rate > birth_rate;

Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


The table shows countries that have a higher death than birth rate.

## Countries that have the highest population/area

In [25]:
%%sql
SELECT name, population, area, population/area
FROM facts
ORDER BY population/area DESC
LIMIT 10;

Done.


name,population,area,population/area
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299


The table consists of the top 10 countries with the highest population/area ratio, with Macau having the highest ratio.